In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dados = pd.read_csv('../input/socioeconomic-country-profiles/soci_econ_country_profiles.csv')
dados.head()

In [ ]:
dados.isnull().sum().values

In [ ]:
categoricos = []
for x in dados:
  if dados[x].dtypes == 'object':
    categoricos.append(x)

In [ ]:
categoricos = dados[categoricos].copy()
categoricos.head()

In [ ]:
categoricos['Pop. using improved drinking water (urban/rural, %)'] = categoricos['Pop. using improved drinking water (urban/rural, %)'].str.split('/')
teste = categoricos['Pop. using improved drinking water (urban/rural, %)'].str.slice(1).sum()

In [ ]:
categoricos['Labour force participation (female/male pop. %)'] = categoricos['Labour force participation (female/male pop. %)'].str.split('/')
categoricos['Labour force participation male %'] = categoricos['Labour force participation (female/male pop. %)'].str.slice(1).sum()
categoricos['Labour force participation female %'] = categoricos['Labour force participation (female/male pop. %)'].str.slice(0,1).sum()
categoricos.drop('Labour force participation (female/male pop. %)', axis=1, inplace=True)

In [ ]:
categoricos['Population age distribution (0-14 / 60+ years, %)'] = categoricos['Population age distribution (0-14 / 60+ years, %)'].str.split('/')
categoricos['Population age distribution 0-14 y.o %'] = categoricos['Population age distribution (0-14 / 60+ years, %)'].str.slice(0,1).sum()
categoricos['Population age distribution 60+ y.o %)'] = categoricos['Population age distribution (0-14 / 60+ years, %)'].str.slice(1).sum()
categoricos.drop('Population age distribution (0-14 / 60+ years, %)', axis=1, inplace=True)

In [ ]:
categoricos['International migrant stock (000/% of total pop.)'] = categoricos['International migrant stock (000/% of total pop.)'].str.split('/')
categoricos['International migrant stock (% of total pop.)'] = categoricos['International migrant stock (000/% of total pop.)'].str.slice(1).sum()
categoricos.drop("International migrant stock (000/% of total pop.)", axis=1, inplace=True)

In [ ]:
categoricos['Education: Primary gross enrol. ratio (f/m per 100 pop.)'] = categoricos['Education: Primary gross enrol. ratio (f/m per 100 pop.)'].str.split('/')
categoricos['Education: Primary gross enrol. ratio (per 100 pop.)'] = categoricos['Education: Primary gross enrol. ratio (f/m per 100 pop.)'].str.slice(0,1).sum()
categoricos.drop('Education: Primary gross enrol. ratio (f/m per 100 pop.)', axis=1, inplace=True)

In [ ]:
categoricos['Education: Secondary gross enrol. ratio (f/m per 100 pop.)'] = categoricos['Education: Secondary gross enrol. ratio (f/m per 100 pop.)'].str.split('/')
categoricos['Education: Secondary gross enrol. ratio (per 100 pop.)'] = categoricos['Education: Secondary gross enrol. ratio (f/m per 100 pop.)'].str.slice(0,1).sum()
categoricos.drop('Education: Secondary gross enrol. ratio (f/m per 100 pop.)', axis=1, inplace=True)

In [ ]:
categoricos['Education: Tertiary gross enrol. ratio (f/m per 100 pop.)'] = categoricos['Education: Tertiary gross enrol. ratio (f/m per 100 pop.)'].str.split('/')
categoricos['Education: Tertiary gross enrol. ratio (per 100 pop.)'] = categoricos['Education: Tertiary gross enrol. ratio (f/m per 100 pop.)'].str.slice(0,1).sum()
categoricos.drop('Education: Tertiary gross enrol. ratio (f/m per 100 pop.)', axis=1, inplace=True)

In [ ]:
categoricos['Forested area (% of land area)'] = categoricos['Forested area (% of land area)'].str.split('/')
categoricos['Forested area (% of land area)'] = categoricos['Forested area (% of land area)'].str.slice(1).sum()

In [ ]:
categoricos['Energy supply per capita (Gigajoules)'] = categoricos['Energy supply per capita (Gigajoules)'].str.split('/')
categoricos['Energy supply per capita (Gigajoules)'] = categoricos['Energy supply per capita (Gigajoules)'].str.slice(0,1).sum()

In [ ]:
categoricos.drop('Pop. using improved drinking water (urban/rural, %)', axis=1, inplace=True)

In [ ]:
categoricos.head()

In [ ]:
categoricos = categoricos.apply(lambda x: x.replace("~0.0", "0"))
categoricos = categoricos.apply(lambda x: x.replace("-~0.0", "0"))
categoricos = categoricos.apply(lambda x: x.replace("...", ""))
categoricos = categoricos.apply(lambda x: x.replace("", 0))

In [ ]:
categoricos.head()

In [ ]:
categoricos.iloc[:,2:] = categoricos.iloc[:,2:].astype('float')

In [ ]:
for x in categoricos.iloc[:,2:].columns:
  categoricos[x] = categoricos[x].replace(0, categoricos[x].mean())
  categoricos[x] = categoricos[x].replace(-99, 0.01)

In [ ]:
categoricos.head()

In [ ]:
categoricos.drop('Pop. using improved sanitation facilities (urban/rural, %)', axis=1,inplace=True)

### Cleaning numeric variables

In [ ]:
numericos = []
for x in dados:
  if dados[x].dtypes != 'object':
    numericos.append(x)

In [ ]:
numericos = dados[numericos].copy()
numericos.head()

In [ ]:
numericos.fillna(numericos.mean(), inplace=True)

### Re-creating the complete data

In [ ]:
dados = pd.concat([categoricos, numericos], axis=1)
dados.head()

### Exploring Region data

In [ ]:
import ipywidgets as widgets

#### Creating a widget to analyze the data

##### Barplot

In [ ]:
grupamento = dados.groupby('Region').mean()

@widgets.interact(Data = grupamento.columns)
def plotagem(Data):
  fig, ax = plt.subplots(figsize=(25,7))
  ax.bar(grupamento[Data].index, height = grupamento[Data])
  ax.axhline(grupamento[Data].mean(), ls='--', color='red', label='Mean')
  ax.set_title(Data)
  ax.legend()

##### Pieplot

In [ ]:
@widgets.interact(Data = grupamento.columns)
def plotagem(Data):
  fig, ax = plt.subplots(figsize=(10,10))
  ax.pie(x=grupamento[Data], labels=grupamento[Data].index)
  ax.set_title(Data, size=20)

### Exploring country data

##### Barplot

In [ ]:
@widgets.interact(Region = dados['Region'], Data = dados.groupby('Region').sum().columns)
def plot_countries(Data, Region):
  dados2 = dados[dados['Region'] == Region]
  dados2.set_index(dados2['country'], inplace=True)
  dados2.drop(['country','Region'], axis=1, inplace=True)
###----------------------------------------------------------
  fig, ax = plt.subplots(figsize=(25,7))
  ax.bar(dados2[Data].index, height = dados2[Data])
  ax.axhline(dados2[Data].mean(), ls='--', color='red', label='Mean')
  ax.set_title(Data)
  ax.legend()

In [ ]:
dados2.set_index(dados['country'], inplace=True)

@widgets.interact(Data = dados2.columns, Region = dados.groupby('Region').mean().index)
def plotagem(Data, Region):
  dados2 = dados[dados['Region'] == Region]
  dados2.set_index(dados2['country'], inplace=True)
  dados2.drop(['country','Region'], axis=1, inplace=True)
###--------------------------------------------------
  fig, ax = plt.subplots(figsize=(10,10))
  ax.pie(x=dados2[Data], labels=dados2[Data].index)
  ax.set_title(Data, size=20)

##### ScatterPlot

In [ ]:
dados2 = dados.groupby('country').head()
dados2.set_index('country', inplace=True)

@widgets.interact(Data = dados2.columns, Data2 = dados2.columns)
def plotagem(Data, Data2):
  dados2 = dados.groupby('country').head()
  dados2.set_index('country', inplace=True)
###--------------------------------------------------
  fig, ax = plt.subplots(figsize=(10,5))
  ax.scatter(x=dados2[Data], y=dados2[Data2])
  ax.set_ylabel("{}".format(Data))
  ax.set_xlabel("{}".format(Data2))
  ax.set_title('Scatter Plot', size=20)


## Clustering

### KMeans algorithm

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
scaler = StandardScaler()
dados_scaled = scaler.fit_transform(dados.loc[:,['High-technology exports (current US$)', 'Local Purchasing Power Index',
                                                 'Unemployment (% of labour force)', 'International trade: Exports (million US$)',
                                                 'International trade: Imports (million US$)',
                                                 'International trade: Balance (million US$)']])

In [ ]:
score = []
range_values = range(1,20)
for i in range_values:
    kmeans = KMeans(n_clusters = i)
    kmeans.fit(dados_scaled)
    score.append(kmeans.inertia_)
    
plt.plot(score, '+-r')

In [ ]:
cluster = KMeans(n_clusters=3)
cluster.fit(dados_scaled)

In [ ]:
previsoes = cluster.labels_
centroides = cluster.cluster_centers_

In [ ]:
previsoes

In [ ]:
pca = PCA(n_components=2)
dados_pca = pca.fit_transform(dados_scaled)

In [ ]:
dados_pca = pd.DataFrame(dados_pca, columns = ['PCA1', 'PCA2'])
dados_pca = pd.concat([dados_pca, pd.DataFrame({"grupo" : previsoes})], axis=1)

In [ ]:
dados_pca2 = pd.concat([dados_pca, dados['country'], dados['Region']], axis=1)

In [ ]:
ax, fig = plt.subplots(figsize=(15,7))
sns.scatterplot(x = dados_pca2['PCA1'], y = dados_pca2['PCA2'], hue = dados_pca2['grupo'], palette  = 'tab10')
plt.title("Cluster")
plt.show()

In [ ]:
dados_pca2[dados_pca2['grupo'] == 2]

In [ ]:
dados_pca2[dados_pca2['grupo'] == 0]

In [ ]:
dados_pca2[dados_pca2['grupo'] == 1]